# NFDRS Chart – Castle Rock with Safe Time Parsing

In [ ]:

import pandas as pd
import requests
from io import StringIO
from datetime import datetime, timedelta
import plotly.graph_objects as go

# ---- CONFIG ----
station_id = "456322"
component = "burningIndex"
component_label = "Burning Index"

# Time ranges
today = datetime.utcnow()
start_hist = "2009-01-01T00:30:00Z"
end_hist = "2019-12-31T23:30:00Z"
start_current = "2025-01-01T00:30:00Z"
end_current = today.strftime("%Y-%m-%dT00:30:00Z")
start_forecast = today.strftime("%Y-%m-%dT00:30:00Z")
end_forecast = (today + timedelta(days=7)).strftime("%Y-%m-%dT00:30:00Z")

# URLs
url_hist = f"https://fems.fs2c.usda.gov/api/climatology/download-nfdr?stationIds={station_id}&startDate={start_hist}&endDate={end_hist}&dataFormat=csv&dataset=all"
url_current = f"https://fems.fs2c.usda.gov/api/climatology/download-nfdr?stationIds={station_id}&startDate={start_current}&endDate={end_current}&dataFormat=csv&dataset=all"
url_forecast = f"https://fems.fs2c.usda.gov/api/climatology/download-nfdr?stationIds={station_id}&startDate={start_forecast}&endDate={end_forecast}&dataFormat=csv&dataset=forecast"

def get_df(url):
    r = requests.get(url)
    if r.ok:
        df = pd.read_csv(StringIO(r.text))
        
if 'observation_time' not in df.columns or df.empty:
    print("⚠️ Dataframe missing 'observation_time' or is empty.")
    return pd.DataFrame()
df['observation_time'] = pd.to_datetime(df['observation_time'], errors='coerce')

        return df[['observation_time', component]].dropna()
    return pd.DataFrame()

# Load datasets
df_hist = get_df(url_hist)
df_current = get_df(url_current)
df_forecast = get_df(url_forecast)

# Simulate climatology percentiles
df_hist['dayofyear'] = df_hist['observation_time'].dt.dayofyear
percentiles = df_hist.groupby('dayofyear')[component].quantile([0.6, 0.8, 0.9, 0.97, 0.999]).unstack()
hist_max = df_hist.groupby('dayofyear')[component].max()
hist_avg = df_hist.groupby('dayofyear')[component].mean()

# Plot setup
fig = go.Figure()


fig.add_trace(go.Scatter(
    x=dates,
    y=percentiles[0.6].values,
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(0,128,0,0.2)',
    name="60th %",
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=dates,
    y=percentiles[0.8].values,
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(144,238,144,0.2)',
    name="80th %",
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=dates,
    y=percentiles[0.9].values,
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(210,180,140,0.2)',
    name="90th %",
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=dates,
    y=percentiles[0.97].values,
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(255,165,0,0.2)',
    name="97th %",
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=dates,
    y=percentiles[0.999].values,
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(255,0,0,0.2)',
    name="99th %",
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=dates,
    y=percentiles[1.0].values,
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(139,0,0,0.2)',
    name="100th %",
    showlegend=False
))
# Historical Max
fig.add_trace(go.Scatter(
    x=dates,
    y=hist_max.values,
    mode='lines',
    name="Max (2009–19)",
    line=dict(color='red'),
    hoverinfo='x+y'
))

# Historical Avg
fig.add_trace(go.Scatter(
    x=dates,
    y=hist_avg.values,
    mode='lines',
    name="Average",
    line=dict(color='gray', dash='dash'),
    hoverinfo='x+y'
))

# Current Obs
fig.add_trace(go.Scatter(
    x=df_current['observation_time'],
    y=df_current[component],
    mode='lines',
    name="2025",
    line=dict(color='black'),
    hoverinfo='x+y'
))

# Forecast
fig.add_trace(go.Scatter(
    x=df_forecast['observation_time'],
    y=df_forecast[component],
    mode='lines',
    name="FCST",
    line=dict(color='black', dash='dot'),
    hoverinfo='x+y'
))

# Layout
fig.update_layout(
    title="Castle Rock – Burning Index<br>Climatology (2009–19) + Obs + Forecast",
    xaxis_title="Date",
    yaxis_title=component_label,
    hovermode="x unified",
    template="plotly_white",
    legend=dict(orientation="h", y=1.05, x=0.5, xanchor="center")
)

fig.show()
